In [26]:
import pandas as pd
import numpy as np

df = pd.read_csv('avy_X-y.csv', index_col='Unnamed: 0')

In [27]:
print(df.columns)
print(len(df))
df.count()

Index(['Temperature_m', 'Relative_Humidity_m', 'Temperature_tl',
       'Relative_Humidity_tl', 'Wind_Speed_Average_tl',
       'Wind_Speed_Maximum_tl', 'Wind_Direction_(deg.)_tl', 'precip', 'depth',
       'pressure', 'above', 'below', 'near', 'storm_slabs', 'wet_slabs',
       'persistent', 'wind_slab', 'loose_wet', 'glide', 'cornices',
       'loose_dry'],
      dtype='object')
28084


Temperature_m               28084
Relative_Humidity_m         28084
Temperature_tl              28084
Relative_Humidity_tl        28084
Wind_Speed_Average_tl       28084
Wind_Speed_Maximum_tl       28084
Wind_Direction_(deg.)_tl    28084
precip                      28084
depth                       28084
pressure                    28084
above                        9910
below                        9910
near                         9910
storm_slabs                  9910
wet_slabs                    9910
persistent                   9910
wind_slab                    9910
loose_wet                    9910
glide                        9910
cornices                     9910
loose_dry                    9910
dtype: int64

In [28]:
df.describe()

,Temperature_m,Relative_Humidity_m,Temperature_tl,Relative_Humidity_tl,Wind_Speed_Average_tl,Wind_Speed_Maximum_tl,Wind_Direction_(deg.)_tl,precip,depth,pressure,...,below,near,storm_slabs,wet_slabs,persistent,wind_slab,loose_wet,glide,cornices,loose_dry
count,28084.000000,28084.000000,28084.000000,28084.000000,28084.000000,28084.000000,28084.000000,28084.000000,28084.000000,28084.000000,...,9910.000000,9910.000000,9910.000000,9910.000000,9910.000000,9910.000000,9910.000000,9910.000000,9910.000000,9910.000000
mean,41.339892,69.057256,40.457058,71.833346,5.993869,13.218412,232.492375,0.010882,35.258536,1016.170093,...,1.932492,2.335116,0.456206,0.031483,0.053280,0.803229,0.519677,0.009687,0.160747,0.036428
std,14.189910,24.539394,14.509618,28.395706,4.859696,8.174939,106.053639,0.034144,42.258102,6.079356,...,0.753626,0.762826,0.498104,0.174629,0.224602,0.397578,0.499638,0.097951,0.367316,0.187362
min,3.229000,4.673000,5.743000,4.038000,-0.036000,0.000000,0.005000,0.000000,0.000000,990.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.880000,48.310000,29.200000,46.875000,1.492750,7.160000,106.300000,0.000000,2.905000,1013.212000,...,1.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,39.380000,76.045000,37.900000,82.500000,5.577500,12.960000,286.700000,0.000000,11.260000,1017.278000,...,2.000000,2.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,50.840000,91.900000,51.105000,99.400000,9.330000,18.380000,310.500000,0.000000,69.307500,1020.165250,...,2.000000,3.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,86.800000,100.000000,85.000000,100.000000,41.900000,72.800000,360.000000,0.740000,143.700000,1034.949000,...,5.000000,5.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Prediction 1:
# Storm Slab

### Storm slab is formed during storms and tends to stabilize very quickly (~3 days after snowfall typical; definitely should resolve within a week)

y = NWAC's forecast of storm slab: 0 or 1 at time T
X = weather conditions, 5 days prior to T 
_possible revision to model: 5 days before and up to 24 hours after (because NWAC may predict storm slab as a hazard when a storm is coming in in the afternoon) - see if this is relevant_

In [33]:
y = df.storm_slabs   # np.npdarray 

In [70]:
# relevant methods for manipluating y

print(y.values.shape)
print(y.count())
print(y.unique())
print(y.iloc[:5].isna())
print('...')
print(y.iloc[-5:].isna())
print()
y_ok = y[~y.isna()]
print('y_ok.__len__() =', len(y_ok))

(28084,)
9910
[ nan   1.   0.]
2014-09-23 09:00:00    True
2014-09-23 10:00:00    True
2014-09-23 11:00:00    True
2014-09-23 12:00:00    True
2014-09-23 13:00:00    True
Name: storm_slabs, dtype: bool
...
2017-12-07 20:00:00    False
2017-12-07 21:00:00    False
2017-12-07 22:00:00    False
2017-12-07 23:00:00    False
2017-12-08 00:00:00    False
Name: storm_slabs, dtype: bool

y_ok.__len__() = 9910


In [36]:
data = df[['Temperature_m', 'Relative_Humidity_m', 'Temperature_tl',
       'Relative_Humidity_tl', 'Wind_Speed_Average_tl',
       'Wind_Speed_Maximum_tl', 'Wind_Direction_(deg.)_tl', 'precip', 'depth',]]
       
       #'above', 'below', 'near', 'storm_slabs', 'wet_slabs',
       #'persistent', 'wind_slab', 'loose_wet', 'glide', 'cornices',
       #'loose_dry'],

In [84]:
print(data.values.shape)
print()
print(data.count())
# ok so: y is 9910 samples; these samples correspond to timestamps from a subset of the data
print()
print(len(data[~y.isna()]), '= number of data corresponding to y_ok')
print()
print(data.loc[y_ok.index[0]])

(28084, 9)

Temperature_m               28084
Relative_Humidity_m         28084
Temperature_tl              28084
Relative_Humidity_tl        28084
Wind_Speed_Average_tl       28084
Wind_Speed_Maximum_tl       28084
Wind_Direction_(deg.)_tl    28084
precip                      28084
depth                       28084
dtype: int64

9910 = number of data corresponding to y_ok

Temperature_m                29.510
Relative_Humidity_m          97.200
Temperature_tl               27.630
Relative_Humidity_tl        100.000
Wind_Speed_Average_tl         2.554
Wind_Speed_Maximum_tl        11.110
Wind_Direction_(deg.)_tl    267.200
precip                        0.000
depth                         9.220
Name: 2014-12-20 00:00:00, dtype: float64


In [98]:
import tqdm  # progress bar

# find i | x[i] matches index of y[j]
# only have to run it once!

dicty = {}    # map from .iloc index of y: corresponding .iloc index of data

lower_limit = 0
for i in tqdm.tqdm(range(len(y_ok))):
    for j in range(lower_limit, len(data)):
        if data.index[j] == y_ok.index[i]:
            dicty[i] = j
            lower_limit = j
            break


100%|██████████| 9910/9910 [00:00<00:00, 84879.28it/s]

In [102]:
dicty[0]

2103

In [104]:
data.values.shape

(28084, 9)

In [153]:
fl = 12  # forward looking constant

X = np.empty((len(y_ok) - fl, 24*7 * 9))   
# 9910 - 30 samples because last day goes into future; 7 days of weather info * 9 columns

for i in range(len(y_ok) - fl):
    X[i] = data.values[dicty[i] - 24*6 - fl: dicty[i] + fl].flatten()
    
# X is now a dataset with 7 days worth of weather for each y

y = y_ok[:-fl]

In [154]:
from sklearn.svm import SVC

my_svm = SVC()

In [156]:
my_svm.fit(X[:-1000], y[:-1000])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [157]:
my_svm.score(X[-1000:], y_ok[-1000:])

0.748

In [159]:
X_test = X[-1000:]
p_test = my_svm.predict(X_test)
y_test = y[-1000:]

correct = 0
for i in range(1000):
    if y_test[i] != p_test[i]:
        print(i, y_test[i], p_test[i])    my_svm.predict(X[-1000:])
    else:
        correct += 1
        
print(correct, 'of 1000')

0 1.0 0.0
1 1.0 0.0
2 1.0 0.0
3 1.0 0.0
4 1.0 0.0
5 1.0 0.0
6 1.0 0.0
7 1.0 0.0
8 1.0 0.0
9 1.0 0.0
10 1.0 0.0
11 1.0 0.0
12 1.0 0.0
13 1.0 0.0
14 1.0 0.0
15 1.0 0.0
16 1.0 0.0
17 1.0 0.0
18 1.0 0.0
19 1.0 0.0
20 1.0 0.0
21 1.0 0.0
22 1.0 0.0
23 1.0 0.0
120 1.0 0.0
121 1.0 0.0
122 1.0 0.0
123 1.0 0.0
124 1.0 0.0
125 1.0 0.0
126 1.0 0.0
127 1.0 0.0
128 1.0 0.0
129 1.0 0.0
130 1.0 0.0
131 1.0 0.0
132 1.0 0.0
133 1.0 0.0
134 1.0 0.0
135 1.0 0.0
136 1.0 0.0
137 1.0 0.0
138 1.0 0.0
139 1.0 0.0
140 1.0 0.0
141 1.0 0.0
142 1.0 0.0
143 1.0 0.0
264 1.0 0.0
265 1.0 0.0
266 1.0 0.0
267 1.0 0.0
268 1.0 0.0
269 1.0 0.0
270 1.0 0.0
271 1.0 0.0
272 1.0 0.0
273 1.0 0.0
274 1.0 0.0
275 1.0 0.0
276 1.0 0.0
277 1.0 0.0
278 1.0 0.0
279 1.0 0.0
280 1.0 0.0
281 1.0 0.0
282 1.0 0.0
283 1.0 0.0
284 1.0 0.0
285 1.0 0.0
286 1.0 0.0
287 1.0 0.0
699 1.0 0.0
700 1.0 0.0
701 1.0 0.0
702 1.0 0.0
703 1.0 0.0
704 1.0 0.0
705 1.0 0.0
706 1.0 0.0
707 1.0 0.0
708 1.0 0.0
709 1.0 0.0
710 1.0 0.0
711 1.0 0.0
712 1.0 0.0
71

In [160]:
y.mean()

0.45675894120024246

In [ ]:
# hmmm...